All Relevant New York and New England Zipcodes to Train Model for EV Adoption Rate

In [ ]:
import pandas as pd

# Get NY and NE zipcode list
df = pd.read_csv('uszips.csv')

state_id = ["CT", "ME", "MA", "NH", "RI", "VT", "NY"]
relevant_zips = df[df["state_id"].isin(state_id)]['zip'].astype(str).str.zfill(5)
relevant_zips.to_csv("training_zips_ne_ny.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'uszips.csv'

## Best Model to Use: ??

Using best random forest model to predict number of evs on the road in each zipcode in New England

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/IEOR 4212 Project/Datasets/Training Data for EV Adoption in New England/TrainTest Data/model_creation_data.csv')
predict_df = pd.read_csv('/content/drive/MyDrive/IEOR 4212 Project/Datasets/Training Data for EV Adoption in New England/TrainTest Data/model_prediction_data.csv')

In [ ]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
predict_df

,zip_code,Estimate Male Total population AGE Under 5 years,Estimate Male Total population AGE 5 to 9 years,Estimate Male Total population AGE 10 to 14 years,Estimate Male Total population AGE 15 to 19 years,Estimate Male Total population AGE 20 to 24 years,Estimate Male Total population AGE 25 to 29 years,Estimate Male Total population AGE 30 to 34 years,Estimate Male Total population AGE 35 to 39 years,Estimate Male Total population AGE 40 to 44 years,...,Estimate Households Median income (dollars),Estimate Households Mean income (dollars),Estimate Families Total,Estimate Families Median income (dollars),Estimate Families Mean income (dollars),Estimate Married-couple families Total,Estimate Married-couple families Median income (dollars),Estimate Nonfamily households Total,Estimate Nonfamily households Median income (dollars),Estimate Nonfamily households Mean income (dollars)
0,1001,430.0,338.0,411.0,417.0,418.0,700.0,436.0,578.0,424.0,...,71924.000000,98266.0,3952,104744.00000,122115.0,2922,118063.000000,2861,46145.000000,58370.000000
1,1002,295.0,465.0,466.0,1141.0,3825.0,749.0,389.0,588.0,491.0,...,69411.000000,111345.0,4331,139955.00000,171271.0,3489,159602.000000,5384,34983.000000,60303.000000
2,1003,0.0,0.0,0.0,3214.0,2334.0,46.0,1.0,0.0,0.0,...,92928.762361,169656.0,24,113481.02782,222854.0,24,127023.467845,10,54029.940274,72576.661823
3,1005,51.0,45.0,398.0,124.0,47.0,276.0,181.0,111.0,84.0,...,97390.000000,106114.0,1260,110603.00000,114870.0,946,125635.000000,428,59559.000000,68449.000000
4,1007,386.0,318.0,538.0,482.0,370.0,420.0,349.0,412.0,488.0,...,99056.000000,113840.0,4382,126993.00000,134035.0,3431,131528.000000,2047,53313.000000,64198.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1780,6902,2170.0,1496.0,1616.0,1903.0,2678.0,4040.0,3205.0,2682.0,2257.0,...,92527.000000,144706.0,17181,99756.00000,154166.0,11020,125185.000000,11211,77518.000000,124359.000000
1781,6903,284.0,451.0,254.0,367.0,505.0,335.0,169.0,372.0,271.0,...,203438.000000,293056.0,4021,230524.00000,319370.0,3714,236944.000000,815,100678.000000,156572.000000
1782,6905,509.0,534.0,625.0,444.0,475.0,946.0,735.0,680.0,614.0,...,132730.000000,166127.0,4782,144018.00000,209597.0,3989,168831.000000,3384,79464.000000,102984.000000
1783,6906,428.0,387.0,282.0,502.0,186.0,319.0,450.0,356.0,452.0,...,91710.000000,130667.0,2389,92436.00000,128272.0,1680,118611.000000,1038,80724.000000,121371.000000


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# Change output to be log
df['EVs on the Road'] = np.log1p(df['EVs on the Road'])

# Train/Test Split
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

df = df.sample(frac=1.0,random_state=42) # Shuffle data
train, test = train_test_split(df, test_size=0.3, random_state=42)

y_train = train['EVs on the Road']
y_test = test['EVs on the Road']

X_train = train.iloc[:, :-1]
X_test = test.iloc[:, :-1]

# Create Model
rf_model = RandomForestRegressor(n_estimators=1000, max_features=15, max_samples=0.9, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(max_features=15, max_samples=0.9, n_estimators=1000,
                      random_state=42)

In [ ]:
# Evaluating Model

# Run Cross-Validation for r_squared
cv_scores = cross_val_score(rf_model, X_train, y_train, cv=5, n_jobs=-1)
print(f"Train R_squared scores: {cv_scores}")
print(f"Train Average R_squared: {cv_scores.mean():.4f}")

# Test metrics
test_rsquare = rf_model.score(X_test, y_test)
test_pred = rf_model.predict(X_test)
test_rmse = mean_squared_error(y_test, test_pred) ** 0.5

print("test r-square", test_rsquare)
print("test rmse", test_rmse)

Train R_squared scores: [0.91773935 0.90206998 0.90551942 0.92674585 0.92835219]
Train Average R_squared: 0.9161
test r-square 0.9116303726606036
test rmse 0.5106679705157239


In [ ]:
predict_df["EVs on the road"] = rf_model.predict(predict_df)

In [ ]:
import numpy as np
predict_df["EVs on the road"] = np.expm1(predict_df["EVs on the road"])

In [ ]:
predict_df = predict_df[["zip_code", "EVs on the road"]]

In [ ]:
predict_df['zip_code'] = predict_df['zip_code'].astype(str)
predict_df['zip_code'] = predict_df['zip_code'].apply(lambda x: str(x).zfill(5))
predict_df['first_three_digits'] = predict_df['zip_code'].str[:3]

zipcode_prefixes = {
    'WCMA': ['010', '011', '012', '013', '014', '015', '016'],
    'NEMA': ['017', '018', '019'],
    'SEMA': ['020', '021', '022', '023', '024', '025', '026', '027'],
    'RI': ['028', '029'],
    'NH': ['030', '031', '032', '033', '034', '035', '036', '037', '038'],
    'ME': ['039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049'],
    'VT': ['050', '051', '052', '053', '054', '056', '057', '058', '059'],
    'CT': ['060', '061', '062', '063', '064', '065', '066', '067', '068', '069']
}

# Invert the dictionary: used AI for this code
prefix_to_region = {
    prefix: region
    for region, prefixes in zipcode_prefixes.items()
    for prefix in prefixes
}

predict_df['Region'] = predict_df['first_three_digits'].map(prefix_to_region)
predict_df

,zip_code,EVs on the road,first_three_digits,Region
0,01001,244.834276,010,WCMA
1,01002,374.552660,010,WCMA
2,01003,50.592740,010,WCMA
3,01005,75.027269,010,WCMA
4,01007,286.978076,010,WCMA
...,...,...,...,...
1780,06902,668.997923,069,CT
1781,06903,646.603554,069,CT
1782,06905,492.513530,069,CT
1783,06906,163.727037,069,CT


In [ ]:
# Proportion of load in each region for each zip code
region_total = predict_df.groupby('Region')['EVs on the road'].transform('sum')
predict_df['proportion_load'] = predict_df['EVs on the road'] / region_total

## Multiply the proportion of ev load in each zipcode by the forecasted load in that region from 2025-2045. Dataset should be 1785*12 months * 20 years


Using the EV Adoption Rate we calculate above, we want to find how the forecasted ev load in new england will fall by region

In [ ]:
## estimate by ev adoption rate
forecast = pd.read_excel('/content/drive/MyDrive/IEOR 4212 Project/Datasets/forecast_data_2025.xlsx', sheet_name="3 Monthly Peaks")

In [ ]:
forecast

,Region,Year,Month,Coincidence,Percentile,Base,EV,HP,Gross,BTMPV,Net
0,ISONE,2025,1,CP,50,19571,117,319,20006,4,20003
1,ISONE,2026,1,CP,50,19591,172,415,20178,1,20177
2,ISONE,2027,1,CP,50,19611,232,623,20467,10,20457
3,ISONE,2028,1,CP,50,19621,358,907,20887,24,20863
4,ISONE,2029,1,CP,50,19636,503,1250,21388,54,21335
...,...,...,...,...,...,...,...,...,...,...,...
2263,WCMA,2041,12,CP,50,2642,541,1068,4251,48,4203
2264,WCMA,2042,12,CP,50,2644,589,1145,4378,67,4311
2265,WCMA,2043,12,CP,50,2645,637,1214,4496,82,4414
2266,WCMA,2044,12,CP,50,2644,687,1272,4603,92,4511


In [ ]:
# Expand predict_df dataframe

years = range(2025, 2046)
months = range(1, 13)

time_df = pd.DataFrame([(y, m) for y in years for m in months],
                       columns=['Year', 'Month'])

# cross join
predict_df = predict_df.merge(time_df, how='cross')

In [ ]:
predict_df = predict_df.merge(
    forecast[['Region', 'Year', 'Month', 'EV']],
    on=['Region', 'Year', 'Month'],
    how='left'
)


In [ ]:
predict_df

,zip_code,EVs on the road,first_three_digits,Region,proportion_load,Year,Month,EV
0,01001,244.834276,010,WCMA,0.008875,2025,1,14
1,01001,244.834276,010,WCMA,0.008875,2025,2,16
2,01001,244.834276,010,WCMA,0.008875,2025,3,15
3,01001,244.834276,010,WCMA,0.008875,2025,4,12
4,01001,244.834276,010,WCMA,0.008875,2025,5,9
...,...,...,...,...,...,...,...,...
449815,06907,243.651364,069,CT,0.003488,2045,8,615
449816,06907,243.651364,069,CT,0.003488,2045,9,640
449817,06907,243.651364,069,CT,0.003488,2045,10,841
449818,06907,243.651364,069,CT,0.003488,2045,11,1303


In [ ]:
predict_df['ev_load'] = predict_df['proportion_load'] * predict_df['EV']
predict_df

,zip_code,EVs on the road,first_three_digits,Region,proportion_load,Year,Month,EV,ev_load
0,01001,244.834276,010,WCMA,0.008875,2025,1,14,0.124245
1,01001,244.834276,010,WCMA,0.008875,2025,2,16,0.141994
2,01001,244.834276,010,WCMA,0.008875,2025,3,15,0.133120
3,01001,244.834276,010,WCMA,0.008875,2025,4,12,0.106496
4,01001,244.834276,010,WCMA,0.008875,2025,5,9,0.079872
...,...,...,...,...,...,...,...,...,...
449815,06907,243.651364,069,CT,0.003488,2045,8,615,2.145286
449816,06907,243.651364,069,CT,0.003488,2045,9,640,2.232493
449817,06907,243.651364,069,CT,0.003488,2045,10,841,2.933635
449818,06907,243.651364,069,CT,0.003488,2045,11,1303,4.545216


In [ ]:
predict_df.groupby(['Region','Year'])['ev_load'].sum()

Region  Year
CT      2025     282.0
        2026     396.0
        2027     530.0
        2028     804.0
        2029    1124.0
                 ...  
WCMA    2041    4767.0
        2042    5181.0
        2043    5588.0
        2044    5983.0
        2045    6376.0
Name: ev_load, Length: 168, dtype: float64

In [ ]:
predict_df['date'] = pd.to_datetime(predict_df['Year'].astype(str) + '-' + predict_df['Month'].astype(str))
predict_df['timestamp'] = predict_df['date'].dt.strftime('%Y-%m-%d') # Folium format
predict_df = predict_df.drop(columns=['date'])

In [ ]:
predict_df.drop(columns=['first_three_digits', 'EV'])

,zip_code,EVs on the road,Region,proportion_load,Year,Month,ev_load
0,01001,244.834276,WCMA,0.008875,2025,1,0.124245
1,01001,244.834276,WCMA,0.008875,2025,2,0.141994
2,01001,244.834276,WCMA,0.008875,2025,3,0.133120
3,01001,244.834276,WCMA,0.008875,2025,4,0.106496
4,01001,244.834276,WCMA,0.008875,2025,5,0.079872
...,...,...,...,...,...,...,...
449815,06907,243.651364,CT,0.003488,2045,8,2.145286
449816,06907,243.651364,CT,0.003488,2045,9,2.232493
449817,06907,243.651364,CT,0.003488,2045,10,2.933635
449818,06907,243.651364,CT,0.003488,2045,11,4.545216


In [ ]:
predict_df.to_csv("/content/drive/MyDrive/IEOR 4212 Project/Datasets/data_to_map.csv")

In [ ]:
predict_df

,zip_code,EVs on the road,first_three_digits,Region,proportion_load,Year,Month,EV,ev_load,date,timestamp
0,01001,244.834276,010,WCMA,0.008875,2025,1,14,0.124245,2025-01-01,2025-01-01
1,01001,244.834276,010,WCMA,0.008875,2025,2,16,0.141994,2025-02-01,2025-02-01
2,01001,244.834276,010,WCMA,0.008875,2025,3,15,0.133120,2025-03-01,2025-03-01
3,01001,244.834276,010,WCMA,0.008875,2025,4,12,0.106496,2025-04-01,2025-04-01
4,01001,244.834276,010,WCMA,0.008875,2025,5,9,0.079872,2025-05-01,2025-05-01
...,...,...,...,...,...,...,...,...,...,...,...
449815,06907,243.651364,069,CT,0.003488,2045,8,615,2.145286,2045-08-01,2045-08-01
449816,06907,243.651364,069,CT,0.003488,2045,9,640,2.232493,2045-09-01,2045-09-01
449817,06907,243.651364,069,CT,0.003488,2045,10,841,2.933635,2045-10-01,2045-10-01
449818,06907,243.651364,069,CT,0.003488,2045,11,1303,4.545216,2045-11-01,2045-11-01
